# Energy Aspects Data Analyst case study
You are a data analyst at Energy Aspects. <br>
The gas team needs a model to help them understand gas demand in Northern Scotland. <br>
They believe Gas demand is tied up to temperatures and tasked you to forecast minimum temperatures in the next 24 months.<br>
For this exercise, you will use weather data from Wick Airport as a proxy for Northern Scotland weather.<br>
The language to be used is Python.<br>
You can use any library you may find useful for the task.<br>

### Case study assessment
- Do not overcomplexify your answers. We value our team's ability to give simple answers to simple questions
- Focus on clean code and short, high level explainations of choices
- Do not spend time tuning models to get accurate results. We will not assess you on model accuracy / tuning
- We will assess relevance of analytics, visualisations, model choice and reasonings

## Data source and description

wick_airport_weather_data.csv<br>
https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data<br>

Wick Airport<br>
Location  336500E 952200N, Lat 58.454 Lon -3.088, 36m amsl<br>
Estimated data is marked with a * after the value.<br>
Missing data (more than 2 days missing in month) is marked by  ---.<br>
Sunshine data taken from an automatic Kipp & Zonen sensor marked with a #, otherwise sunshine data taken from a Campbell Stokes recorder.<br>
- Mean daily maximum temperature (tmax): degC
- Mean daily minimum temperature (tmin): degC
- Days of air frost (af): days
- Total rainfall (rain): mm
- Total sunshine duration (sun): hours

# Import data
Import data from data/wick_airport_weather_data.csv

# Extract and transform
Make all the cleaning and transformation steps you judge necessary to analyse the data

# Data exploration and charting
Perform Exploratory Data Analysis on the dataset.<br>
Present your findings in a few lines and charts.

# Modelling
Choose a model, backtest it and generate forecasts for the 24 months ahead.<br>
Explain your model choice in a few lines and present backtesting / forecasting results using tables and charts understandable for analysts who do not write code or build statistical models.
